# Examples
 
Note: Please make sure that trimesh, numpy, matplotlib are installed. You can use pip to install them:

```
pip install "trimesh[easy]" numpy matplotlib
```

In [ ]:
import trimesh
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Flatten a 3D surface mesh while minimizing the area distortion locally

This is a wrapper around cgal; read more about the `Discrete Authalic Parameterization` here:
https://doc.cgal.org/latest/Surface_mesh_parameterization/index.html#title8

In [ ]:
mesh = trimesh.Trimesh(
    vertices=[[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0], [0, 0, 1]],
    faces=[[4, 0, 1], [4, 1, 2], [4, 2, 3], [4, 3, 0]],
    face_colors=np.array([[255, 0, 0, 100], [0, 255, 0, 0], [0, 0, 255, 0], [255, 0, 255, 0]], dtype=np.uint8))

In [ ]:
 # note: this does not render in GitHub; run the notebook locally to be able to interactively look at a pyramid
mesh.show()

Now we can create a `SurfaceMesh` object, and populate it with the mesh:

In [ ]:
from cgal_pybind import Point_3, SurfaceMesh
surface_mesh = SurfaceMesh()
surface_mesh.add_vertices([Point_3(v[0], v[1], v[2]) for v in mesh.vertices])
surface_mesh.add_faces([tuple(f) for f in mesh.faces])

Then we can generate the flat points after the authalic transformation:

In [ ]:
flat_points = np.array(surface_mesh.authalic()[0])
flat_points

... and plot them:

In [ ]:
plt.scatter(flat_points[:, 0], flat_points[:, 1])

## Compute the streamlines intersection points with the surface separating two layers

Create layer and direction vector dataset:

In [ ]:
layers = np.zeros((20, 3, 3), dtype=np.uint8)
for i in range(4):
    layers[i * 5 : (i + 1) * 5, ...] = i + 1
layers = np.pad(layers, 2, "constant", constant_values=0)

direction_vectors = np.full(layers.shape + (3,), np.nan, dtype=np.float32)
direction_vectors[2:-2, 2:-2, 2:-2] = [1.0, 0.0, 0.0]

Display voxel shape; note that the boundary between 2 & 3 is the interface where `blue` meets `green`

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
colors = np.empty_like(layers, dtype=object)
colors[layers == 1] = 'red'
colors[layers == 2] = 'blue'
colors[layers == 3] = 'green'
colors[layers == 4] = 'white'

ax.voxels(layers > 0, facecolors=colors, edgecolor='k');


Based on the directions, let's find the intersection points:

In [ ]:
from cgal_pybind import compute_streamlines_intersections

voxel_to_point_map = compute_streamlines_intersections(
    layers=layers,
    offset=np.array([1.0, 2.0, 3.0], dtype=np.float32),
    voxel_dimensions=np.array([2.0, 2.0, 2.0], dtype=np.float32),
    vector_field=direction_vectors,
    layer_1=np.uint8(2),
    layer_2=np.uint8(3),
)

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
points = voxel_to_point_map[~np.isnan(voxel_to_point_map)].reshape((-1, 3))
ax.scatter(points[:, 0], points[:, 1], points[:, 2]);